# Chronos T5 


In [2]:
import yfinance as yf


data = yf.download('BTC-USD', interval='1d')
data.to_csv('BTC-USD.csv')


[*********************100%%**********************]  1 of 1 completed


In [ ]:
import pandas as pd 

data = pd.read_csv('BTC-USD.csv', parse_dates=['Date'], index_col=['Date'])
data.drop(['Adj Close', 'Open', 'High', 'Low', 'Volume'], axis=1, inplace=True)


In [14]:
import ta

data['sma'] = ta.trend.sma_indicator(data['Close'], window=30)
data['ema20'] = ta.trend.ema_indicator(data['Close'], window=20)
data['ema50'] = ta.trend.ema_indicator(data['Close'], window=50)
data['ema200'] = ta.trend.ema_indicator(data['Close'], window=200)


# Añadir RSI
data['rsi'] = ta.momentum.rsi(data['Close'], window=14)

# Añadir MACD
macd = ta.trend.MACD(data['Close'], window_slow=26, window_fast=12, window_sign=9)
data['macd'] = macd.macd()
data['macd_signal'] = macd.macd_signal()

#Precio Futuro
data['Close_future'] = data['Close'].shift(-1)
# Dirección del Movimiento de Precio
data['Price_Up'] = (data['Close_future'] > data['Close']).astype(int)


#Boramos datos nulos
data.dropna(inplace=True)

data.to_csv('BTC-Indicators.csv')


In [13]:
data.dtypes

Close          float64
sma            float64
ema20          float64
ema50          float64
ema200         float64
rsi            float64
macd           float64
macd_signal    float64
dtype: object